In [52]:
# Importo todas las librerías

import pandas as pd
import numpy as np
import folium
from folium import plugins
import pymongo
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [27]:
# Aquí llamo a la base de datos y a la colección de companies

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client.companies
collection_companies = db.companies

In [28]:
# Aquí creo mi primer query en la cual busco únicamente empresas que tengan un giro financiero

query_dict = list(collection_companies.find({'category_code': 'finance'}))

In [29]:
# Aquí creo una nueva base de datos y una nueva colección

client2 = pymongo.MongoClient('mongodb://localhost:27017/')
db2 = client2.finance_db
collection_finance = db2.finance_collection

In [30]:
# Aquí inserto mi query a la nueva colección

collection_finance.insert_many(query_dict)

In [137]:
# En esta celda creo un DataFrame con origen de una Query. La Query en primera instancia no podía ser normalizada por json_normalize ya que era una lista de listas de dicionarios, por lo tanto use una list comprehension para crear la lista de diccionarios y poder llegar a un DataFrame con las coordenadas y el nombre de la empresa

df = pd.DataFrame(collection_finance.find({}, {'name': 1, 'offices.latitude': 1, 'offices.longitude': 1, '_id': 0}))
lst = [df.offices[i][0] for i in range(len(df.offices))]
array = np.asarray(lst)
df.offices = array
df2 = json_normalize(df.offices)
df2['name'] = df['name']
df2 = df2[['name', 'latitude', 'longitude']]
df3 = df2.dropna().reset_index(drop=True)

In [160]:
# Aquí creo un mapa y luego un loop que agregue marcadores para cada una de las empresas con etiqueta

mapa = folium.Map()
for i in range(len(df3.latitude)):
    folium.CircleMarker([df3.latitude[i], df3.longitude[i]]).add_child(folium.Popup(df3['name'][i])).add_to(mapa)

In [161]:
# En este último paso agrego un marcador con etiqueta con la decisión final del lugar ideal para establecer la startup

folium.Marker([40.78343, -73.96625], icon=folium.Icon(color='green')).add_child(folium.Popup('Ideal spot for office')).add_to(mapa)
mapa.save('office.html')